In [0]:
csv_path = "dbfs:/FileStore/shared_uploads/letebrhan@gmail.com/sales_data.csv"  # change to your path

df = spark.read.option("header", "true").csv(csv_path, inferSchema=True)
df.show(5)


+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156|2016-11-08|2016-11-11|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset 

In [0]:
import re

# Clean and rename all column names
df_clean = df.toDF(*[re.sub(r'[^A-Za-z0-9_]', '', c.replace(' ', '_')) for c in df.columns])
df_clean.printSchema()


root
 |-- Row_ID: integer (nullable = true)
 |-- Order_ID: string (nullable = true)
 |-- Order_Date: date (nullable = true)
 |-- Ship_Date: date (nullable = true)
 |-- Ship_Mode: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Customer_Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal_Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- SubCategory: string (nullable = true)
 |-- Product_Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)



In [0]:
# Drop rows with null sales or order dates
df_clean = df_clean.dropna(subset=["Sales", "Order_Date"])


In [0]:
# Add Derived Columns

from pyspark.sql.functions import to_date, month, year

df_clean = df_clean.withColumn("Order_Date", to_date("Order_Date", "MM/dd/yyyy"))
df_clean = df_clean.withColumn("Order_Year", year("Order_Date"))
df_clean = df_clean.withColumn("Order_Month", month("Order_Date"))


In [0]:
# Save Cleaned DataFrame Temporarily
df_clean.write.mode("overwrite").parquet("/tmp/superstore_cleaned")


In [0]:
# Step 1: Load the Cleaned Data
df_clean = spark.read.parquet("/tmp/superstore_cleaned")

# Step 2: Save as Delta Table
df_clean.write.format("delta").mode("overwrite").save("/tmp/delta_superstore")

#  Step 3: Read and Display
df_delta = spark.read.format("delta").load("/tmp/delta_superstore")
df_delta.show(5)



+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+-----------+--------------------+--------+--------+--------+--------+----------+-----------+
|Row_ID|      Order_ID|Order_Date| Ship_Date|     Ship_Mode|Customer_ID|  Customer_Name|  Segment|      Country|           City|     State|Postal_Code|Region|     Product_ID|       Category|SubCategory|        Product_Name|   Sales|Quantity|Discount|  Profit|Order_Year|Order_Month|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+-----------+--------------------+--------+--------+--------+--------+----------+-----------+
|     1|CA-2016-152156|2016-11-08|2016-11-11|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420|

In [0]:
# (Optional) Step 4: Time Travel

df_old = spark.read.format("delta").option("versionAsOf", 0).load("/tmp/delta_superstore")
df_old.show()


+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+-----------+--------------------+--------+--------+--------+--------+----------+-----------+
|Row_ID|      Order_ID|Order_Date| Ship_Date|     Ship_Mode|Customer_ID|     Customer_Name|    Segment|      Country|           City|         State|Postal_Code| Region|     Product_ID|       Category|SubCategory|        Product_Name|   Sales|Quantity|Discount|  Profit|Order_Year|Order_Month|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+-----------+--------------------+--------+--------+--------+--------+----------+-----------+
|     1|CA-2016-152156|2016-11-08|2016-11-11|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|    

In [0]:
# Goal:
# Use PySpark and SQL to explore sales trends, and visualize key business metrics.
#  Step 1: Load the Delta Table
df = spark.read.format("delta").load("/tmp/delta_superstore")
df.createOrReplaceTempView("superstore")


In [0]:
df = spark.read.format("delta").load("/tmp/delta_superstore")
df.createOrReplaceTempView("superstore")
df.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+-----------+--------------------+--------+--------+--------+--------+----------+-----------+
|Row_ID|      Order_ID|Order_Date| Ship_Date|     Ship_Mode|Customer_ID|     Customer_Name|    Segment|      Country|           City|         State|Postal_Code| Region|     Product_ID|       Category|SubCategory|        Product_Name|   Sales|Quantity|Discount|  Profit|Order_Year|Order_Month|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+-----------+--------------------+--------+--------+--------+--------+----------+-----------+
|     1|CA-2016-152156|2016-11-08|2016-11-11|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|    

In [0]:
df = spark.read.format("delta").load("/tmp/delta_superstore")
df.printSchema()

df.createOrReplaceTempView("superstore")
df.show(3)

root
 |-- Row_ID: integer (nullable = true)
 |-- Order_ID: string (nullable = true)
 |-- Order_Date: date (nullable = true)
 |-- Ship_Date: date (nullable = true)
 |-- Ship_Mode: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Customer_Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal_Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- SubCategory: string (nullable = true)
 |-- Product_Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)
 |-- Order_Year: integer (nullable = true)
 |-- Order_Month: integer (nullable = true)

+------+--------------+----------+----------+------------+-----

In [0]:
%sql
SELECT * FROM superstore LIMIT 5


Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,SubCategory,Product_Name,Sales,Quantity,Discount,Profit,Order_Year,Order_Month
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136,2016,11
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.582,2016,11
3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0,6.8714,2016,6
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.031,2015,10
5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368,2,0.2,2.5164,2015,10


In [0]:
%sql
SELECT Category, ROUND(SUM(Sales), 2) AS TotalSales
FROM superstore
GROUP BY Category
ORDER BY TotalSales DESC


Category,TotalSales
Technology,835900.07
Furniture,733046.86
Office Supplies,703502.93
